In [1]:
import os
import tensorflow as tf
import math
import numpy as np
import itertools
from google.cloud import storage

tf.enable_eager_execution()

from waymo_open_dataset.utils import range_image_utils
from waymo_open_dataset.utils import transform_utils
from waymo_open_dataset.utils import  frame_utils
from waymo_open_dataset import dataset_pb2 as open_dataset
from object_detection.utils import dataset_util
from waymo_open_dataset import label_pb2

In [2]:
bucket_name = 'waymo_validation'
storage_client = storage.Client()
bucket = storage_client.get_bucket(bucket_name)


In [3]:
def create_tf_example(frame,count):
  # TODO(user): Populate the following variables from your example.
  center_x = [] # List of normalized left x coordinates in bounding box (1 per box)
  center_y = [] # List of normalized right x coordinates in bounding box
             # (1 per box)
  width = [] # List of normalized top y coordinates in bounding box (1 per box)
  height = [] # List of normalized bottom y coordinates in bounding box
             # (1 per box)
  classes_text = [] # List of string class name of bounding box (1 per box)
  classes = [] # List of integer class id of bounding box (1 per box)
  
  filename = bytes(('waymo_training_front_'+str(count)+'.jpg'), 'utf-8') # Filename of the image. Empty if image is not from file
  #row = []
  
  for camera_image in frame.images:
    if camera_image.name == 1:
      #row.append(filename.decode('utf-8'))
      encoded_image_data = camera_image.image
      #img_writer = open('validation_dataset/images/'+filename.decode('utf-8'),'wb')
      #img_writer.write(encoded_image_data)
      
      #blob = bucket.blob('validation_dataset/images/'+filename.decode('utf-8'))

      #blob.upload_from_string(encoded_image_data,content_type='image/png')
      
      #img_writer.close()
      break
  for camera_labels in frame.camera_labels:
    if camera_labels.name == 1:
      for label in camera_labels.labels:
        if label.type == label_pb2.Label.Type.TYPE_VEHICLE or label.type == label_pb2.Label.Type.TYPE_PEDESTRIAN:

          center_x.append(label.box.center_x)
          center_y.append(label.box.center_y)
          width.append(label.box.length)
          height.append(label.box.width)
            
#           x_min = label.box.center_x - 0.5 * label.box.length
#           y_min = label.box.center_y - 0.5 * label.box.width
#           x_max = label.box.center_x + 0.5 * label.box.length
#           y_max = label.box.center_y + 0.5 * label.box.width
            
          if label.type == label_pb2.Label.Type.TYPE_VEHICLE:
            classes.append(0)
            classes_text.append(b"vehicle")
            label = 0
            
          else:
            classes.append(1)
            classes_text.append(b"pedestrian")
            label = 1
#           row.append(x_min)
#           row.append(y_min)
#           row.append(x_max)
#           row.append(y_max)
#           row.append(label)
        
            
#          row = row + ',' + str(x_min) + ',' + str(y_min) + ',' + str(x_max) + ',' + str(y_max) + ',' + str(label)


  #print(encoded_image_data)  

  image_format = b'jpeg'
  #plt.imshow(tf.image.decode_jpeg(encoded_image_data), cmap=None)
 # print(dataset_util.bytes_feature(tf.image.decode_jpeg(encoded_image_data)))  
 
  tf_example = tf.train.Example(features=tf.train.Features(feature={
      'image/filename': dataset_util.bytes_feature(filename),
      'image/format': dataset_util.bytes_feature(image_format),
      'image/encoded': dataset_util.bytes_feature(encoded_image_data),
      'image/object/bbox/center_x': dataset_util.float_list_feature(center_x),
      'image/object/bbox/center_y': dataset_util.float_list_feature(center_y),
      'image/object/bbox/width': dataset_util.float_list_feature(width),
      'image/object/bbox/height': dataset_util.float_list_feature(height),
      'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
      'image/object/class/label': dataset_util.int64_list_feature(classes),
  }))
  #csv_writer.writerow(row)
  return tf_example

In [9]:
"""image_feature_description={
      'image/filename': tf.io.VarLenFeature(tf.string),
      'image/format': tf.io.FixedLenFeature([],tf.string),
      'image/encoded': tf.io.FixedLenFeature([],tf.string),
      'image/object/bbox/center_x': tf.io.VarLenFeature(tf.float32),
      'image/object/bbox/center_y': tf.io.VarLenFeature(tf.float32),
      'image/object/bbox/width': tf.io.VarLenFeature(tf.float32),
      'image/object/bbox/height': tf.io.VarLenFeature(tf.float32),
      'image/object/class/text': tf.io.VarLenFeature(tf.string),
      'image/object/class/label': tf.io.VarLenFeature(tf.int64),
  }"""

In [8]:
filenames = [os.path.join('data/', f) for f in os.listdir('data/') if 'tfrecord' in f]

In [9]:
FILENAME = filenames
#FILENAME = 'data/segment-10448102132863604198_472_000_492_000_with_camera_labels.tfrecord'
#csvFile = open('validation_dataset/csv_files/validation_annotation.csv', 'w')
#csv_writer = csv.writer(csvFile)
#writer.writerow(row)   
    
dataset = tf.data.TFRecordDataset(FILENAME, compression_type='')
count = 1
frame = open_dataset.Frame()
file_count = 24
record_file = 'gs://waymo_validation/Training_data/tf_records/waymo_images.tfrecords'
writer = tf.python_io.TFRecordWriter(record_file+str(file_count).zfill(5))
for data in dataset:
    if count % 200 == 0:
        writer.close()
        file_count += 1
        writer = tf.python_io.TFRecordWriter(record_file+str(file_count).zfill(5))
    frame.ParseFromString(bytearray(data.numpy()))
    example = create_tf_example(frame,count)
    writer.write(example.SerializeToString())
    count += 1
    
    #print(example)
    #if count == 3:
    

writer.close()
#csvFile.close()
print(file_count)
count

47


4764

In [34]:
input_record_files = [os.path.join('validation_dataset/', f) for f in os.listdir('validation_dataset/') if 'tfrecord' in f]

def _parse_function(proto):
    # define your tfrecord again. Remember that you saved your image as a string.
    image_feature_description={
      'image/filename': tf.io.VarLenFeature(tf.string),
      'image/format': tf.io.FixedLenFeature([],tf.string),
      'image/encoded': tf.io.FixedLenFeature([],tf.string),
      'image/object/bbox/center_x': tf.io.VarLenFeature(tf.float32),
      'image/object/bbox/center_y': tf.io.VarLenFeature(tf.float32),
      'image/object/bbox/width': tf.io.VarLenFeature(tf.float32),
      'image/object/bbox/height': tf.io.VarLenFeature(tf.float32),
      'image/object/class/text': tf.io.VarLenFeature(tf.string),
      'image/object/class/label': tf.io.VarLenFeature(tf.int64),
  }
    # Load one example
    parsed_features = tf.parse_single_example(proto, image_feature_description)
    
    # Turn your saved image string into an array
    # parsed_features['image/encoded'] = tf.decode_raw(
    #     parsed_features['image/encoded'], tf.uint8)
    parsed_features['image/encoded'] = tf.io.decode_jpeg(
    parsed_features['image/encoded'])
    
    return parsed_features['image/encoded'], parsed_features["image/object/class/text"]

In [42]:
#dataset = tf.data.TFRecordDataset(input_record_files)
dataset = ds_train
# Maps the parser on every filepath in the array. You can set the number of parallel loaders here
dataset = dataset.map(_parse_function, num_parallel_calls=8)

# This dataset will go on forever
dataset = dataset.repeat()

# Set the number of datapoints you want to load and shuffle 
dataset = dataset.shuffle(3)

# Set the batchsize
dataset = dataset.batch(8)

# Create an iterator
iterator = dataset.make_one_shot_iterator()

# Create your tf representation of the iterator
image, label = iterator.get_next()
# Bring your picture back in shape

print(image.shape)
#print(image)


#image = tf.reshape(image, [-1, 1280, 1920, 3])
#print(image)
# Create a one hot array for your labels
#label = tf.one_hot(label, NUM_CLASSES)

(8, 1280, 1920, 3)


In [36]:
!ls validation_dataset/

Untitled.ipynb		     waymo_images.tfrecords00012
waymo_images.tfrecords00000  waymo_images.tfrecords00013
waymo_images.tfrecords00001  waymo_images.tfrecords00014
waymo_images.tfrecords00002  waymo_images.tfrecords00015
waymo_images.tfrecords00003  waymo_images.tfrecords00016
waymo_images.tfrecords00004  waymo_images.tfrecords00017
waymo_images.tfrecords00005  waymo_images.tfrecords00018
waymo_images.tfrecords00006  waymo_images.tfrecords00019
waymo_images.tfrecords00007  waymo_images.tfrecords00020
waymo_images.tfrecords00008  waymo_images.tfrecords00021
waymo_images.tfrecords00009  waymo_images.tfrecords00022
waymo_images.tfrecords00010  waymo_images.tfrecords00023
waymo_images.tfrecords00011  waymo_images.tfrecords00024


In [41]:
ds_train = tf.data.TFRecordDataset('gs://waymo_validation/validation_dataset/waymo_images.tfrecords00000')

In [43]:
os.listdir('gs://waymo_validation/validation_dataset')

FileNotFoundError: [Errno 2] No such file or directory: 'gs://waymo_validation/validation_dataset'